[View in Colaboratory](https://colab.research.google.com/github/gr3ybr0w/cookbook/blob/master/Machine_Learning/Neural_Networks/Implementing_Gradient_Descent.ipynb)

In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv('https://stats.idre.ucla.edu/stat/data/binary.csv')

In [4]:
data.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [0]:
# Make dummy variables for rank
data = pd.get_dummies(data, columns=['rank'])

In [0]:
# Standarize features
for field in ['gre', 'gpa']:
    mean, std = data[field].mean(), data[field].std()
    data.loc[:,field] = (data[field]-mean)/std

In [0]:
# Split off random 10% of the data for testing
np.random.seed(42)
sample = np.random.choice(data.index, size=int(len(data)*0.9), replace=False)
data, test_data = data.loc[sample], data.drop(sample)

In [0]:
# Split into features and targets
features, targets = data.drop('admit', axis=1), data['admit']
features_test, targets_test = test_data.drop('admit', axis=1), test_data['admit']

In [0]:
def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

In [0]:
# Use to same seed to make debugging easier
np.random.seed(42)

In [0]:
n_records, n_features = features.shape
last_loss = None

In [20]:
# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)
weights

array([ 0.64471093,  0.31330392, -0.19166212,  0.22149921, -0.18918948,
       -0.19013338])

In [0]:
# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

In [22]:
for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Activation of the output unit
        #   Notice we multiply the inputs and the weights here 
        #   rather than storing h as a separate variable 
        output = sigmoid(np.dot(x, weights))

        # The error, the target minus the network output
        error = y - output

        # The error term
        #   Notice we calulate f'(h) here instead of defining a separate
        #   sigmoid_prime function. This just makes it faster because we
        #   can re-use the result of the sigmoid function stored in
        #   the output variable
        error_term = error * output * (1 - output)

        # The gradient descent step, the error times the gradient times the inputs
        del_w += error_term * x

    # Update the weights here. The learning rate times the 
    # change in weights, divided by the number of records to average
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

Train loss:  0.2457295175893598
Train loss:  0.2092533843881859
Train loss:  0.20357937984778354
Train loss:  0.20184070465272508
Train loss:  0.2011439385987615
Train loss:  0.20081746433746886
Train loss:  0.20064790956345346
Train loss:  0.20055325963437892
Train loss:  0.20049758534792156
Train loss:  0.2004635519153199


In [23]:
# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

Prediction accuracy: 0.750
